# Indoorコンペ　1週間チャレンジ

<br>Indoorコンペに1週間限定で参加し、銅メダルを獲得した解法を残します。
<br>コンペ詳細はこちら。https://www.kaggle.com/c/indoor-location-navigation

## PipeLine

1. アンサンブル
<br>LB最高スコアのアンサンブルと、MultiOutputMLPのアンサンブルが最もscoreが高かったため採用
<br>以降の処理は、アンサンブルの結果に対して後処理をするパートとなる。

2. device id Leak
<br>アンサンブルの結果に対してdevice id leakegeを適用。参照先は下記のURL。
<br>https://www.kaggle.com/iwatatakuya/use-leakage-considering-device-id-postprocess

3. Snap to grid
<br>上記の結果に対して、snap to gridを適用。（閾値を外れている推定位置を最近傍のGridに移動させる。）参照先は下記のURL。
<br>https://www.kaggle.com/robikscube/indoor-navigation-snap-to-grid-post-processing



# アンサンブルパート

In [1]:
import multiprocessing
import numpy as np
import pandas as pd
import scipy.interpolate
import scipy.sparse
from tqdm import tqdm

import json
import re
import gc
import pickle
import itertools
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime as dt
from pathlib import Path
from tqdm import tqdm
import datetime

# from indoor_location_competition_20.io_f import read_data_file
# import indoor_location_competition_20.compute_f as compute_f

from dataclasses import dataclass

import warnings # Supress warnings 
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt

from tqdm.notebook import tqdm

import json
from datetime import datetime
from glob import glob

In [9]:

exp1 = pd.read_csv('../input/indoor-emsemble/exp1_multioutput_mlp.csv')
exp2 = pd.read_csv('../input/indoor-emsemble/exp2_Cost_Minim.csv').drop('index',axis = 1)
exp3 = pd.read_csv('../input/indoor-emsemble/exp3 use_leakage_considering.csv')
exp4 = pd.read_csv('../input/indoor-emsemble/exp2_Cost_Minim.csv').drop('index',axis = 1)
exp5 = pd.read_csv('../input/indoor-emsemble/exp5_ensemble.csv')

LB4555 = pd.read_csv('../input/lb4555/submission.csv')
LB4522 = pd.read_csv('../input/lb4522/submission.csv')
LB4722 = pd.read_csv('../input/lb4722/submission.csv')
LB4470 = pd.read_csv('../input/indoor-loc-and-nav-subs/submission_4470.csv')
LB4470emsemble = pd.read_csv('../input/indoor-emsemble/LB4470_0.95exp2_0.05.csv')
LB4483 = pd.read_csv('../input/indoor-loc-and-nav-subs/submission_4_483.csv')

leak_4555 = pd.read_csv('../input/indoor-emsemble/submission_df_leak_end_LB4555.csv')
leak_4556 = pd.read_csv('../input/indoor-emsemble/submission_df_leak_start_LB4556.csv')
leak_4725 = pd.read_csv('../input/indoor-emsemble/submission_df_leak_all_LB4725.csv')


In [13]:
df = pd.merge(exp2,leak_4555,how = 'inner', on = 'site_path_timestamp')
df = df.merge(LB4522,how = 'inner' ,on = 'site_path_timestamp')

col_dict = {'x_x':'sub0_x',
          'y_x':'sub0_y',
          'x_y':'sub1_x',
          'y_y':'sub1_y',
          'x':'sub2_x',
          'y':'sub2_y',
          }
df = df.rename(columns=col_dict)

In [14]:

# # ver12
df_emsemble = pd.DataFrame(LB4522)

LB4470 = LB4470.set_index('site_path_timestamp')
exp2 = exp2.set_index('site_path_timestamp')

df_emsemble['x'] = 0.8*LB4470['x'].values + 0.2*exp2.loc[LB4470.index, 'x'].values
df_emsemble['y'] = 0.8*LB4470['y'].values + 0.2*exp2.loc[LB4470.index, 'y'].values


# # # ver11
# # df = LB4522.copy
# LB4470 = LB4470.set_index('site_path_timestamp')
# exp2 = exp2.set_index('site_path_timestamp')

# LB4470['x'] = 0.85*LB4470['x'].values + 0.15*exp2.loc[LB4470.index, 'x'].values
# LB4470['y'] = 0.85*LB4470['y'].values + 0.15*exp2.loc[LB4470.index, 'y'].values


# # # ver11
# # df = LB4522.copy
# LB4470 = LB4470.set_index('site_path_timestamp')
# exp2 = exp2.set_index('site_path_timestamp')

# LB4470['x'] = 0.9*LB4470['x'].values + 0.1*exp2.loc[LB4470.index, 'x'].values
# LB4470['y'] = 0.9*LB4470['y'].values + 0.1*exp2.loc[LB4470.index, 'y'].values



# # # ver11
# # df = LB4522.copy
# LB4470 = LB4470.set_index('site_path_timestamp')
# exp2 = exp2.set_index('site_path_timestamp')

# LB4470['x'] = 0.95*LB4470['x'].values + 0.05*exp2.loc[LB4470.index, 'x'].values
# LB4470['y'] = 0.95*LB4470['y'].values + 0.05*exp2.loc[LB4470.index, 'y'].values

# # # ver9
# # df = LB4522.copy
# LB4470 = LB4470.set_index('site_path_timestamp')
# LB4483 = LB4483.set_index('site_path_timestamp')

# LB4522 = LB4522.set_index('site_path_timestamp')
# LB4555 = LB4555.set_index('site_path_timestamp')
# exp4 = exp4.set_index('site_path_timestamp')

# #ver10
# LB4470['x'] = 0.7*LB4470['x'].values + 0.3*LB4483.loc[LB4470.index, 'x'].values
# LB4470['y'] = 0.7*LB4470['y'].values + 0.3*LB4483.loc[LB4470.index, 'y'].values

# # ver9 
# LB4522['x'] = 0.4*LB4522['x'].values + 0.4*LB4555.loc[LB4522.index, 'x'].values + 0.2*exp4.loc[LB4522.index, 'x'].values
# LB4522['y'] = 0.4*LB4522['y'].values + 0.4*LB4555.loc[LB4522.index, 'y'].values + 0.2*exp4.loc[LB4522.index,'y'].values

# # # ver8
# df['x'] = df['sub0_x']*0.4 + df['sub1_x']*0.3 + df['sub2_x']*0.3
# df['y'] = df['sub0_y']*0.4 + df['sub1_y']*0.3 + df['sub2_y']*0.3


# # ver7
# df['x'] = df['sub1_x']*0.4 + df['sub2_x']*0.6
# df['y'] = df['sub1_y']*0.4 + df['sub2_y']*0.6

# # exp6
# df['x'] = df['sub0_x']*0.4 + df['sub1_x']*0.3 + df['sub2_x']*0.3
# df['y'] = df['sub0_y']*0.4 + df['sub1_y']*0.3 + df['sub2_y']*0.3

In [15]:
df_emsemble

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,93.213821,96.479153
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,83.239300,99.883803
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,84.942722,105.075408
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,86.081008,109.093861
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,87.944170,112.365452
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,208.630347,101.163545
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,206.774168,103.814537
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,204.091932,110.622288
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,196.018451,116.925300


In [16]:
# LB4522 = LB4522.reset_index(drop=False)
df_emsemble.to_csv('LB4470*08+exp2*02.csv', index=True)

# device Leakege 実装パート
https://www.kaggle.com/iwatatakuya/use-leakage-considering-device-id-postprocess


In [ ]:
df_sample = df_emsemble.copy()

df_sample["site_id"] = df_emsemble["site_path_timestamp"].apply(lambda x:x.split('_')[0])
df_sample["path_id"] = df_sample["site_path_timestamp"].apply(lambda x:x.split('_')[1])
df_sample["timestamp"] = df_sample["site_path_timestamp"].apply(lambda x:x.split('_')[2]).astype(int)
list_site = df_sample["site_id"].unique()

In [ ]:
@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray
    starttime: int
    endtime: int

def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []
    starttime = -1
    endtime = -1

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data: continue
        if len(line_data) > 20: 
            if line_data[2:7] == 'start':
                if len(line_data.split(':')) > 1: starttime = int(line_data.split(':')[1])
                else: starttime = int(line_data.split('\t')[2])
                continue   
            if line_data[2:5] == 'end':
                if len(line_data.split(':')) > 1: endtime = int(line_data.split(':')[1])
                else: endtime = int(line_data.split('\t')[2])
                continue 

        if line_data[0] == '#': continue

        line_data = line_data.split('\t')
        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])
            continue
       
        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        
        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        
        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        
        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            if len(line_data) > 7:
                magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4]), float(line_data[5]), float(line_data[6]), float(line_data[7])])
            else:
                magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4]), 0, 0, 0])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            dist = line_data[7]
            last_seen_timestamp = line_data[9]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi, dist,last_seen_timestamp]
            ibeacon.append(ibeacon_data)
            continue
        
    
    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)
    
    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint, starttime, endtime)

In [ ]:
df_leak = pd.DataFrame()

print(f'list_site Length : {len(list_site)}')

for site_id in tqdm(list_site):
    print(site_id)
    
    ## train
    list_train_files = glob(f"../input/indoor-location-navigation/train/{site_id}/**/*.txt", recursive = True)
    l_pid = []
    l_sts = []
    l_ets = []
    l_swpx = []
    l_swpy = []
    l_ewpx = []
    l_ewpy = []
    l_d = []
    for filename in tqdm(list_train_files):
        path_id = filename.split(".")[2].split("/")[6]
        try: df_all = read_data_file(filename)
        except:continue
        
        # get start and end timestamp
        sts = df_all.starttime
        ets = df_all.endtime
        
        df_waypoint = pd.DataFrame(df_all.waypoint)
        df_waypoint.columns = ['timestamp', 'waypoint_x','waypoint_y']
        df_waypoint["timestamp"] = (df_waypoint["timestamp"]).astype(float)
        
        # search start and end waypoints of each path
        swpx = df_waypoint.query("timestamp == @df_waypoint.timestamp.min()")["waypoint_x"].iloc[0]
        swpy = df_waypoint.query("timestamp == @df_waypoint.timestamp.min()")["waypoint_y"].iloc[0]
        ewpx = df_waypoint.query("timestamp == @df_waypoint.timestamp.max()")["waypoint_x"].iloc[0]
        ewpy = df_waypoint.query("timestamp == @df_waypoint.timestamp.max()")["waypoint_y"].iloc[0]

        # search device id besed on magn bias
        d = sum(df_all.magn_uncali[0,4:7])
        if d == 0:d = sum(df_all.magn_uncali[0,1:4] - df_all.magn[0,1:4])
        d = round(d,2)

        l_pid += [path_id]
        l_sts += [sts]
        l_ets += [ets]
        l_swpx += [swpx]
        l_swpy += [swpy]
        l_ewpx += [ewpx]
        l_ewpy += [ewpy]
        l_d += [d]        
    df_mart_train = pd.DataFrame(data={"path_id": l_pid,
                                       "start_time": l_sts, "end_time": l_ets,
                                       "start_waypoint_x": l_swpx, "start_waypoint_y": l_swpy,
                                       "end_waypoint_x": l_ewpx, "end_waypoint_y": l_ewpy,
                                       "device": l_d},
                                 columns=["path_id","start_time","end_time","start_waypoint_x","start_waypoint_y",
                                          "end_waypoint_x", "end_waypoint_y", "device"])

    l_pid = []
    l_sts = []
    l_ets = []
    l_swpx = []
    l_swpy = []
    l_ewpx = []
    l_ewpy = []
    l_d = []
    
    ## test
    df_sample_site = df_sample.query("site_id == @site_id")
    df_sample_site["timestamp"] = df_sample_site["timestamp"].astype(float)
    list_path = df_sample_site["path_id"].unique()
    for path_id in tqdm(list_path):
        df_sample_path = df_sample_site.query("path_id == @path_id")
        filename = f"../input/indoor-location-navigation/test/{path_id}.txt"
        df_all = read_data_file(filename)
        df_wifi = pd.DataFrame(df_all.wifi)
        df_wifi.columns = ['timestamp', 'ssid', 'bssid', 'rssi', 'last_seen_timestamp']
        df_wifi["timestamp"] = (df_wifi["timestamp"]).astype(float)
        df_wifi["last_seen_timestamp"] = (df_wifi["last_seen_timestamp"]).astype(float)  
        
        df_ibeacon = pd.DataFrame(df_all.ibeacon)
        # retrieve raw timestamp
        if len(df_ibeacon) > 0:
            df_ibeacon.columns = ['timestamp', 'uuid', 'rssi', 'dist','last_seen_timestamp']
            df_ibeacon["timestamp"] = (df_ibeacon["timestamp"]).astype(float)
            df_ibeacon["last_seen_timestamp"] = (df_ibeacon["last_seen_timestamp"]).astype(float)
            time_diff = df_ibeacon.loc[0,"last_seen_timestamp"]-df_ibeacon.loc[0,"timestamp"]
        else:
            time_diff = (df_wifi["last_seen_timestamp"] - df_wifi["timestamp"]).max()
            
        # search device id besed on magn bias
        d = sum(df_all.magn_uncali[0,4:7])
        if d == 0:d = sum(df_all.magn_uncali[0,1:4] - df_all.magn[0,1:4])
        d = round(d,2)
        
        sts = df_all.starttime + time_diff
        ets = df_all.endtime + time_diff
        swpx = np.nan;swpy = np.nan; ewpx = np.nan; ewpy = np.nan;floor = np.nan
        # x and y
        df_start = df_mart_train.query("device == @d and start_time > @ets - 2000 and start_time < @ets + 10000").sort_values("start_time").reset_index()
        df_end = df_mart_train.query("device == @d and end_time < @sts + 2000 and end_time > @sts - 10000").sort_values("end_time",ascending=False).reset_index()
        if len(df_start) > 0:
            ewpx = df_start.iloc[0]["start_waypoint_x"]
            ewpy = df_start.iloc[0]["start_waypoint_y"]
        if len(df_end) > 0:
            swpx = df_end.iloc[0]["end_waypoint_x"]
            swpy = df_end.iloc[0]["end_waypoint_y"]

        l_pid += [path_id]
        l_sts += [sts]
        l_ets += [ets]
        l_swpx += [swpx]
        l_swpy += [swpy]
        l_ewpx += [ewpx]
        l_ewpy += [ewpy]
        l_d += [d]    
      
    df_mart_test = pd.DataFrame(data={"path_id": l_pid,
                                       "start_time": df_all.starttime, "end_time": df_all.endtime,
                                       "start_waypoint_x": l_swpx, "start_waypoint_y": l_swpy,
                                       "end_waypoint_x": l_ewpx, "end_waypoint_y": l_ewpy,
                                       "device": l_d},
                                 columns=["path_id","start_time","end_time","start_waypoint_x","start_waypoint_y",
                                          "end_waypoint_x", "end_waypoint_y","device"])
    
    df_leak = df_leak.append(df_mart_test)
    
    # calculate time difference and waypoint difference
    df_tr = df_mart_train.sort_values(["device","start_time"]).reset_index(drop=True)
    df_tr["time_diff"] = df_tr["start_time"] - df_tr.groupby("device").shift(1)["end_time"]
    df_tr["x_diff"] = df_tr["start_waypoint_x"] - df_tr.groupby("device").shift(1)["end_waypoint_x"]
    df_tr["y_diff"] = df_tr["start_waypoint_y"] - df_tr.groupby("device").shift(1)["end_waypoint_y"]
    
    print(f'{site_id} : Done')

In [ ]:
df_sample

In [ ]:
# apply leakage waypoint
df_device_leak = df_sample.copy()
list_path = df_device_leak["path_id"].unique()

In [ ]:

for path_id in tqdm(list_path):
    df_device_leak_path = df_device_leak.query("path_id == @path_id")
    start_idx = df_device_leak.loc[df_device_leak["path_id"] == path_id].index.min()
    end_idx = df_device_leak.loc[df_device_leak["path_id"] == path_id].index.max()
    start_x = df_device_leak_path.at[start_idx,"x"]
    start_y = df_device_leak_path.at[start_idx,"y"]
    end_x = df_device_leak_path.at[end_idx,"x"]
    end_y = df_device_leak_path.at[end_idx,"y"]
    start_x_leak = df_leak.query("path_id == @path_id")["start_waypoint_x"].iloc[0]
    start_y_leak = df_leak.query("path_id == @path_id")["start_waypoint_y"].iloc[0]
    end_x_leak = df_leak.query("path_id == @path_id")["end_waypoint_x"].iloc[0]
    end_y_leak = df_leak.query("path_id == @path_id")["end_waypoint_y"].iloc[0]
    if not np.isnan(start_x_leak):
        df_device_leak.at[start_idx,"x"] = start_x_leak
        df_device_leak.at[start_idx,"y"] = start_y_leak
    if not np.isnan(end_x_leak):
        df_device_leak.at[end_idx,"x"] = end_x_leak
        df_device_leak.at[end_idx,"y"] = end_y_leak

In [ ]:
df_device_leak.drop(["site_id","path_id","timestamp"],axis=1).to_csv("LB4470_085_exp2_015_device_leak.csv",index = False)

# Snap to Grid パート
https://www.kaggle.com/robikscube/indoor-navigation-snap-to-grid-post-processing

In [ ]:
def split_col(df):
    df = pd.concat([
        df['site_path_timestamp'].str.split('_', expand=True) \
        .rename(columns={0:'site',
                         1:'path',
                         2:'timestamp'}),
        df
    ], axis=1).copy()
    return df

floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}


def plot_preds(
    site,
    floorNo,
    sub=None,
    true_locs=None,
    base="../input/indoor-location-navigation",
    show_train=True,
    show_preds=True,
    fix_labels=True,
    map_floor=None
):
    """
    Plots predictions on floorplan map.
    
    map_floor : use a different floor's map
    """
    if map_floor is None:
        map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)

    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    floor_img = plt.imread(f"{base}/metadata/{site}/{map_floor}/floor_image.png")

    fig, ax = plt.subplots(figsize=(12, 12))
    plt.imshow(floor_img)

    if show_train:
        true_locs = true_locs.query('site == @site and floorNo == @map_floor').copy()
        true_locs["x_"] = true_locs["x"] * floor_img.shape[0] / height_meter
        true_locs["y_"] = (
            true_locs["y"] * -1 * floor_img.shape[1] / width_meter
        ) + floor_img.shape[0]
        true_locs.query("site == @site and floorNo == @map_floor").groupby("path").plot(
            x="x_",
            y="y_",
            style="+",
            ax=ax,
            label="train waypoint location",
            color="grey",
            alpha=0.5,
        )

    if show_preds:
        sub = sub.query('site == @site and floorNo == @floorNo').copy()
        sub["x_"] = sub["x"] * floor_img.shape[0] / height_meter
        sub["y_"] = (
            sub["y"] * -1 * floor_img.shape[1] / width_meter
        ) + floor_img.shape[0]
        for path, path_data in sub.query(
            "site == @site and floorNo == @floorNo"
        ).groupby("path"):
            path_data.plot(
                x="x_",
                y="y_",
                style=".-",
                ax=ax,
                title=f"{site} - floor - {floorNo}",
                alpha=1,
                label=path,
            )
    if fix_labels:
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(
            by_label.values(), by_label.keys(), loc="center left", bbox_to_anchor=(1, 0.5)
        )
    return fig, ax

def sub_process(sub, train_waypoints):
    train_waypoints['isTrainWaypoint'] = True
    sub = split_col(sub[['site_path_timestamp','floor','x','y']]).copy()
    sub = sub.merge(train_waypoints[['site','floorNo','floor']].drop_duplicates(), how='left')
    sub = sub.merge(
        train_waypoints[['x','y','site','floor','isTrainWaypoint']].drop_duplicates(),
        how='left',
        on=['site','x','y','floor']
             )
    sub['isTrainWaypoint'] = sub['isTrainWaypoint'].fillna(False)
    return sub.copy()


def plot_hist(sub):
    sub['dist_pp_change'] = np.sqrt(((sub['x'] - sub['_x_']) ** 2) + ((sub['y'] - sub['_y_']) ** 2))
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    sub['dist_pp_change'].plot(kind='hist', bins=30,
                               ax=axs[0],
                               title='Distance Changed by Post Processing')
    sub.query('dist_pp_change > 0.1')['dist_pp_change'] \
        .plot(kind='hist', bins=30, ax=axs[1],
              title='Distance Changed (Excluding <0.1 Change)')
    
    plt.show()

In [ ]:
# sub = pd.read_csv('../input/indoor-emsemble/LB4470_09_exp2_01_device_leak.csv')

train_waypoints = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')
# sub = sub_process(pd.read_csv('../input/indoor-location-train-waypoints/6.578LB_submission.csv'),
#                  train_waypoints)
sub = sub_process(df_device_leak, train_waypoints)
# Plot the training Data For an example Floor
example_site = '5dbc1d84c1eb61796cf7c010'
example_floorNo = 'F3'

plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds=False)
plt.show()

In [ ]:
from scipy.spatial.distance import cdist

def add_xy(df):
    df['xy'] = [(x, y) for x,y in zip(df['x'], df['y'])]
    return df

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

sub = add_xy(sub)
train_waypoints = add_xy(train_waypoints)

In [ ]:
ds = []
for (site, myfloor), d in sub.groupby(['site','floor']):
    true_floor_locs = train_waypoints.loc[(train_waypoints['floor'] == myfloor) &
                                          (train_waypoints['site'] == site)] \
        .reset_index(drop=True)
    if len(true_floor_locs) == 0:
        print(f'Skipping {site} {myfloor}')
        continue
    d['matched_point'] = [closest_point(x, list(true_floor_locs['xy'])) for x in d['xy']]
    d['x_'] = d['matched_point'].apply(lambda x: x[0])
    d['y_'] = d['matched_point'].apply(lambda x: x[1])
    ds.append(d)

sub = pd.concat(ds)

# Example of raw predictions
plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds=True)
plt.show()

In [ ]:
th = 4

sub_th4 = sub.copy()

def snap_to_grid(sub, threshold):
    """
    Snap to grid if within a threshold.
    
    x, y are the predicted points.
    x_, y_ are the closest grid points.
    _x_, _y_ are the new predictions after post processing.
    """
    sub['_x_'] = sub['x']
    sub['_y_'] = sub['y']
    sub.loc[sub['dist'] < threshold, '_x_'] = sub.loc[sub['dist'] < threshold]['x_']
    sub.loc[sub['dist'] < threshold, '_y_'] = sub.loc[sub['dist'] < threshold]['y_']
    return sub.copy()

# Calculate the distances
sub_th4['dist'] = np.sqrt( (sub_th4.x-sub_th4.x_)**2 + (sub_th4.y-sub_th4.y_)**2 )

sub_pp_th4= snap_to_grid(sub_th4, threshold = th)

sub_pp_th4 = sub_pp_th4[['site_path_timestamp','floor','_x_','_y_','site','path','floorNo']] \
    .rename(columns={'_x_':'x', '_y_':'y'})

# Plot example after post processing
plot_preds(example_site, example_floorNo, sub_pp_th4,
           train_waypoints, show_preds=True)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))

sub_th4['dist'].plot(kind='hist', bins=30,title='Distance Changed by Post Processing')

In [ ]:
plot_hist(sub_th4)

In [ ]:
sub_pp_th4[['site_path_timestamp','floor','x','y']] \
    .to_csv('LB4470_08_exp2_02_thresh4_snap_to_grid.csv', index=False)
# /LB4470_0.95exp2_0.05_device_leak